# BUILD THE NEURAL NETWORK

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


## Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([0])


## Model Layers

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [8]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [9]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.2378,  0.3959, -0.0453, -0.5258, -0.7879, -0.0882, -0.1610,  0.1740,
         -0.3363,  0.3792,  0.3704, -0.4959, -0.0644, -0.2682, -0.0025,  0.0723,
         -0.1705,  0.5638,  0.2875,  0.0106],
        [ 0.2506,  0.5016, -0.2852, -0.2832, -0.5455, -0.1632, -0.5646,  0.3492,
         -0.3513,  0.1962,  0.2156, -0.8093, -0.1993, -0.2909, -0.0135,  0.2279,
          0.0932,  0.0533,  0.2286, -0.2353],
        [-0.0970,  0.3893, -0.0921, -0.4023, -0.4506,  0.1074, -0.1894,  0.1043,
         -0.1062,  0.1635, -0.1036, -0.5961,  0.1867, -0.1476, -0.3819,  0.6009,
          0.1465,  0.4387,  0.0231, -0.3121]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2378, 0.3959, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1740, 0.0000,
         0.3792, 0.3704, 0.0000, 0.0000, 0.0000, 0.0000, 0.0723, 0.0000, 0.5638,
         0.2875, 0.0106],
        [0.2506, 0.5016, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3492, 0.0000,
         0.1962, 0.2156, 0.0000, 0.0000, 0.0000, 0.00

### nn.Sequential

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [11]:
logits

tensor([[ 0.1656,  0.0158,  0.2265,  0.2237, -0.0410, -0.2263, -0.0361, -0.0287,
          0.1452, -0.1404],
        [ 0.2265,  0.0473,  0.2132,  0.1789, -0.0224, -0.2906, -0.1362, -0.0201,
          0.0483, -0.0987],
        [ 0.1524,  0.0484,  0.1870,  0.1751,  0.0223, -0.1707, -0.0979, -0.0476,
          0.0822, -0.1478]], grad_fn=<AddmmBackward0>)

### nn.Softmax

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
pred_probab

tensor([[0.1133, 0.0975, 0.1204, 0.1200, 0.0921, 0.0765, 0.0926, 0.0933, 0.1110,
         0.0834],
        [0.1221, 0.1021, 0.1205, 0.1164, 0.0952, 0.0728, 0.0850, 0.0954, 0.1022,
         0.0882],
        [0.1132, 0.1021, 0.1172, 0.1158, 0.0994, 0.0820, 0.0882, 0.0927, 0.1056,
         0.0839]], grad_fn=<SoftmaxBackward0>)

## Model Parameters

In [14]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0089, -0.0260,  0.0054,  ..., -0.0171, -0.0252,  0.0333],
        [-0.0292, -0.0148, -0.0045,  ...,  0.0288,  0.0234,  0.0043]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0031, -0.0108], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0074,  0.0368, -0.0204,  ..., -0.0267, -0.0425, -0.0388],
        [-0.0397, -0.0281, -0.0008,  ...,  0.0179,  0.0340,  0.0118]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 